# Get Started with ReSEn

Welcome to the Reusable Software Environment (ReSEn) platform!  In this tutorial, we will introduce you to many of the packages installed in ReSEn that are specifically useful for geospace science.  We are only going go go over basic use cases here - for detailed explination of packages, please see the tutorial/documentation specific to that package.

## visuamisr
visuamisr is a python toolkit for accessing and visualizing data from the Advanced Modular Incoherent Scatter Radars (AMISR).

In [4]:
import visuamisr as va

ModuleNotFoundError: No module named 'visuamisr'

## mangopy
mangopy is a python toolkit for accessing and visualizing images from the Midlatitude Allsky-imaging Network for GeoSpace Observations (MANGO).

In [5]:
import mangopy as mango

ModuleNotFoundError: No module named 'mangopy'

## davitpy
davitpy is the DAta VIsualization Toolkit and it is designed to work primarially with the Super Dual Auroral Radar Network (SuperDARN).  SuperDARN consists of over 30 HF ionospheric radars distributed between the northern and southern hemispheres.
<img src='files/Screenshots/superdarn_fov.png'>

In [6]:
import davitpy as sd

ModuleNotFoundError: No module named 'davitpy'

# apexpy
apexpy is a python wrapper around the FORTRAN implimentation of the Apex magnetic coordinate system.  It allows for converting between geodetic, apex, and quasidipole magnetic coordinates, as described by Laundal and Richmond, 2016.

In [10]:
from apexpy import Apex

# Initialize an instance of the Apex class with the year
A = Apex(date=2019.1)

# convert geodetic coordinates to modified apex coordinates
mlat, mlon = A.convert(60,75,'geo','apex',height=300)

# convert modified apex coordinates to geodetic coordinates
glat, glon = A.convert(65,100,'apex','geo',height=300)

# find the magnetic local time of a particular magnetic longitude at a given UT
import datetime as dt
mlt = A.mlon2mlt(120, dt.datetime(2019, 2, 10, 18, 0, 0))

ModuleNotFoundError: No module named 'apexpy'

# spacepy
spacepy is a python package that contains many utilities useful for working with space-based data sources.

In [1]:
import spacepy as space

# time conversion
# coordinate conversion
# omni
# pyCDF?

ModuleNotFoundError: No module named 'spacepy'